In [7]:
a = '2\u20130'
print(a)

2–0


In [9]:
a.encode('ascii', 'ignore').decode()

'20'

In [8]:
a

'2–0'

In [ ]:

predicts = torch.tensor([[1.0000e+00, 1.5581e-14, 8.0536e-15, 6.5220e-14, 9.3445e-14]],
       device='cuda:0')

In [31]:

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import time
from elasticsearch7 import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
import json
import time
import re
glue_text = open('glue_text.txt', 'w')
start = time.time()
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def elastic_more_like_this(sentence):
    res  = ""
    a = es.search(index='bulk1', query = {
        "more_like_this" : {
            "fields" : ['text'],
            "like" : f"{sentence}",
            "min_term_freq" : 1,
            "max_query_terms" : 12
            }
        }
    )
    count = 0
    res_list = []
    for element in a['hits']['hits']:
        el = element['_source']['text'].strip()
        #отделить знаки препинания от текста
        re_el = re.sub(r'[]!"$%&\'()*+,./:;=#@?[\\^_`{|}~-]+', r' \g<0> ', el).strip()
        res = f' {re_el} '
        res_list.append(res)

    return res_list

In [34]:
sentence = 'Syria : 24 - Salem Bitar , 3 - Bachar Srour ; 4 - Hassan Abbas , 5 - Tarek Jabban , 6 - Ammar Awad ( 9 - Louay Taleb 69 ) , 8 - Nihad al-Boushi , 10 - Mohammed Afash , 12 - Ali Dib , 13 - Abdul Latif Helou ( 17 - Ammar Rihawiy 46 ) , 14 - Khaled Zaher ; 16 - Nader Jokhadar .'

In [35]:
elastic_more_like_this(sentence)

[]

In [30]:
a = es.search(index='bulk1', query = {
        "more_like_this" : {
            "fields" : ['text'],
            "like" : f"{sentence}",
            "min_term_freq" : 1,
            "max_query_terms" : 12
            }
        }
    )
a

{'took': 6,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 19.296719,
  'hits': [{'_index': 'bulk1',
    '_type': '_doc',
    '_id': '899786',
    '_score': 19.296719,
    '_ignored': ['text.keyword'],
    '_source': {'text': 'Initially, officials were unable to contact the Wolong National Nature Reserve, home to around 280 giant pandas. However, the Foreign Ministry later said that a group of 31 British tourists visiting the Wolong Panda Reserve in the quake-hit area returned safe and uninjured to Chengdu. Nonetheless, the well-being of an even greater number of pandas in the neighbouring panda reserves remained unknown. Five security guards at the reserve were killed by the earthquake. Six pandas escaped after their enclosures were damaged. By May 20, two pandas at the reserve were found to be injured, while the search continued for another two adult pandas that went mis

In [26]:
input_file = open("spanner.test")
import pprint
min_res = 100
max_res = 0

res_list = []
conll = json.load(input_file)
count_sentence = 0
for sentence in conll:
    count_sentence += 1
    if count_sentence %100 == 0:
        print(f'count_sentence = {count_sentence} time = {time.time() - start}')
        #break

    span_posLabel = sentence['span_posLabel']
    context = sentence['context']
    #print(f'context = {context}')
    a = es.search(index='bulk1', query = {
        "more_like_this" : {
            "fields" : ['text'],
            "like" : f"{sentence}",
            "min_term_freq" : 1,
            "max_query_terms" : 12
            }
        }
    )
    #pprint.pprint(a)
    
    res = a['took']
    #print(res)
    abc = a['hits']['hits']
    count_answers = len(abc)
    #print(f'len = {len(abc)}')
    for element in a['hits']['hits']:
        el = element['_source']['text'].strip()
        #отделить знаки препинания от текста
        re_el = re.sub(r'[]!"$%&\'()*+,./:;=#@?[\\^_`{|}~-]+', r' \g<0> ', el).strip()
        #print('re_el = ', re_el)

    if count_answers < min_res:
        min_res = count_answers
        print(f'context = {context}\n min_res = {min_res}')
    if count_answers > max_res:
        max_res = count_answers
        print(f'context = {context}\n max_res = {max_res}')

context = -DOCSTART-
 min_res = 10
context = -DOCSTART-
 max_res = 10
context = China controlled most of the match and saw several chances missed until the 78th minute when Uzbek striker Igor Shkvyrin took advantage of a misdirected defensive header to lob the ball over the advancing Chinese keeper and into an empty net .
 min_res = 7
context = Takuya Takagi scored the winner in the 88th minute , rising to head a Hiroshige Yanagimoto cross towards the Syrian goal which goalkeeper Salem Bitar appeared to have covered but then allowed to slip into the net .
 min_res = 3
context = Syria : 24 - Salem Bitar , 3 - Bachar Srour ; 4 - Hassan Abbas , 5 - Tarek Jabban , 6 - Ammar Awad ( 9 - Louay Taleb 69 ) , 8 - Nihad al-Boushi , 10 - Mohammed Afash , 12 - Ali Dib , 13 - Abdul Latif Helou ( 17 - Ammar Rihawiy 46 ) , 14 - Khaled Zaher ; 16 - Nader Jokhadar .
 min_res = 0
count_sentence = 100 time = 1763.7061443328857
count_sentence = 200 time = 1764.4930102825165
count_sentence = 300 time = 1765

KeyboardInterrupt: 